In [ ]:
import openpifpaf

# Overview

Plugins are Python modules that extend the functionality of OpenPifPaf. Plugins are developed independently of OpenPifPaf (i.e. you don't need to clone and modify OpenPifPaf's core code).

This is an overview of OpenPifPaf's main components and which are extendible with plugins:
![plugin overview](images/plugins_overview.png)

## Discovery

Plugins are modules whose name starts with `openpifpaf_`. Some plugins are also included in `openpifpaf.contrib`. During the discovery process, every plugin's `register()` function is called. A simple plugin that does nothing, the `nothingplugin` would have this file structure:

```
|- openpifpaf_nothingplugin  # project directory (root directory for version control)
  |- Readme.md
  |- openpifpaf_nothingplugin  # Python module
    |- __init__.py
```

where the `__init__.py` file would contain an empty `register()` function:

```py
def register():
    pass
```

## Registrations

A plugin contains new implementations of base classes provided by OpenPifPaf. The `register()` function adds these implementations to OpenPifPaf's lists of implementations. Those lists are:

In [ ]:
openpifpaf.DATAMODULES

In [ ]:
openpifpaf.DECODERS

In [ ]:
openpifpaf.BASE_TYPES

In [ ]:
openpifpaf.BASE_FACTORIES

In [ ]:
openpifpaf.HEAD_TYPES

In [ ]:
openpifpaf.HEAD_FACTORIES

The relevant base classes that must be derived from to extend the above lists are `DataModule` (in {ref}`api:datasets`), `decoder.Generator` (in {ref}`api:decoder`), `BaseNetwork` and `HeadNetwork` (both in {ref}`api:network`).

## Headmeta

Headmeta (see {ref}`api:headmeta`) is a class that holds configuration data about a head network. It is instantiated in a DataModule (above) and used throughout OpenPifPaf to configure various other parts. For example, the `cocokp` head meta instances are: 

In [ ]:
openpifpaf.datasets.CocoKp().head_metas[0]

In [ ]:
openpifpaf.datasets.CocoKp().head_metas[1]

When a new network is created, information from the head metas will be used to create the appropriate torch graph for the heads. It will use the type of the head meta (`openpifpaf.headmeta.Cif`, `openpifpaf.headmeta.Caf`, ...) and information like the number of keypoints in Cif or the number of skeleton connections in Caf to know how many feature maps to create.

Similarly, the decoder will look for heads that are of type `headmeta.Cif` and `headmeta.Caf` to instantiate the CifCaf decoder.

To get started, have a look how the head metas are created in `CocoKp` for new pose datasets and in `CocoDet` for new detection datasets.

(coordinate-system)=
## Coordinate System

When adding your own datasets or transformations, it is good to keep in mind the coordinate system that OpenPifPaf assumes. The short summary is, everything is done in such a way that annotation coordinates only need to be multiplied by the image rescaling factor without offset. Therefore, the (0, 0) feature map needs to be aligned with the (0, 0) pixel.

![coordinate system](images/coordinate_system.png)

This also shows that training image sizes must be a multiple of the network stride _plus one_, i.e., in the above image, an input image size of $5\times5$ would be optimal for a feature map of size $2\times2$ because the stride is 4.